<a href="https://colab.research.google.com/github/RoaaM/SemEval_competion/blob/main/task_A(binary_classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 64.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 KB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 64.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py

In [ ]:
!pip install Unidecode

In [ ]:
!pip install wordninja

In [ ]:
import ktrain
from ktrain import text
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/train_all_tasks.csv')
df = df[['text', 'label_sexist']]
df.head()

,text,label_sexist
0,"Damn, this writing was pretty chaotic",not sexist
1,"Yeah, and apparently a bunch of misogynistic v...",not sexist
2,How the FUCK is this woman still an MP!!!???,not sexist
3,Understand. Know you're right. At same time I ...,not sexist
4,Surprized they didn't stop and rape some women,not sexist


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['label_sexist'] = le.fit_transform(df['label_sexist'])
df.head()



,text,label_sexist
0,"Damn, this writing was pretty chaotic",0
1,"Yeah, and apparently a bunch of misogynistic v...",0
2,How the FUCK is this woman still an MP!!!???,0
3,Understand. Know you're right. At same time I ...,0
4,Surprized they didn't stop and rape some women,0


In [ ]:
df.shape

(14000, 2)

In [ ]:
df['label_sexist'].value_counts()

0    10602
1     3398
Name: label_sexist, dtype: int64

In [ ]:
X = df['text'].tolist()
y = df['label_sexist'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_array(x_train = X_train,
                                                                      y_train = y_train,
                                                                      x_test = X_test,
                                                                      y_test = y_test,
                                                                      class_names = ['0', '1'],
                                                                      preprocess_mode = 'bert',
                                                                      ngram_range = 1,
                                                                      maxlen = 320)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [ ]:
# define the classifier
model = text.text_classifier('bert',
                              train_data = (X_train, y_train),
                              preproc = preproc)

Is Multi-Label? False
maxlen is 320
done.


In [ ]:
learner = ktrain.get_learner(model,
                             train_data = (X_train, y_train),
                             batch_size = 8)

In [ ]:
history = learner.fit_onecycle(1e-5, 2)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/2
1400/1400 [==============================] - 845s 593ms/step - loss: 0.4507 - accuracy: 0.7962
Epoch 2/2
1400/1400 [==============================] - 832s 594ms/step - loss: 0.2734 - accuracy: 0.8871


In [ ]:
learner.validate(val_data = (X_test, y_test))

88/88 [==============================] - 72s 773ms/step
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      2115
           1       0.81      0.68      0.74       685

    accuracy                           0.88      2800
   macro avg       0.86      0.81      0.83      2800
weighted avg       0.88      0.88      0.88      2800



array([[2006,  109],
       [ 221,  464]])

In [ ]:
# now make predict on test file
test = pd.read_csv('/content/dev_task_a_entries.csv')
predictor = ktrain.get_predictor(learner.model, preproc)
test_result=pd.Series(predictor.predict(test_) for test_ in test['text'])

1/1 [==============================] - 0s 55ms/step


In [ ]:
predictions = list(test_result)
label = ['not sexist' if x == 0 else 'sexist' for x in predictions]

In [ ]:
test_csv= pd.DataFrame({'rewire_id':test['rewire_id'], 'label_pred':label})
test_csv.head()

,rewire_id,label_pred
0,sexism2022_english-11999,0
1,sexism2022_english-9895,0
2,sexism2022_english-11623,0
3,sexism2022_english-19035,0
4,sexism2022_english-895,0


In [ ]:
test_csv['label_pred'].value_counts()

sexist    2000
Name: label_pred, dtype: int64

In [ ]:
test_csv.to_csv('TESTING_RESULT.csv', index=False)